In [1]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import praw
from nltk.sentiment import SentimentIntensityAnalyzer
import os
import spacy

app = Flask(__name__)
CORS(app)

# Initialize PRAW
reddit = praw.Reddit(client_id='srDUoJOjy8WI8ul2CCUiUQ',
                     client_secret='p0ZGHduBE9bdeB0mSt-0o4kSfw4BsQ',
                     user_agent='web:my_reddit_analyzer:v1.0 (by /u/BellaGiulia)')

# Initialize VADER
sia = SentimentIntensityAnalyzer()

nlp = spacy.load("en_core_web_sm")

@app.route('/reddit_sentiment', methods=['GET'])
def reddit_sentiment():
    offset = int(request.args.get('offset', 0))
    hot_posts = reddit.subreddit('TrueOffMyChest').hot(params={"after": offset}, limit=10)
    sentiments = []

    for post in hot_posts:
        analysis = sia.polarity_scores(post.title)
        content_analysis = sia.polarity_scores(post.selftext)
        sentiments.append({
            'title': post.title,
            'positive': analysis['pos'],
            'negative': analysis['neg'],
            'neutral': analysis['neu'],
            'compound': analysis['compound'],
            'score': str(analysis),
            'content': post.selftext,
            'content_sentiment': str(content_analysis),
            'content_sentiment2': content_analysis['compound'],
        })

    return jsonify(sentiments)

@app.route('/search_subreddits', methods=['GET'])
def search_subreddits():
    topic = request.args.get('topic', '')
    comment_limit = request.args.get('comment_limit', 10, type=int)  # Default to 10 comments
    search_results = reddit.subreddit('all').search(topic, limit=10)
    posts_data = []

    for post in search_results:
        post.comments.replace_more(limit=0)  # Load all comment threads
        comments = []
        for comment in post.comments[:comment_limit]:  # Fetch only the first few comments
            doc = nlp(comment.body)
            entities = [(ent.text, ent.label_) for ent in doc.ents]
            sentiment = sia.polarity_scores(comment.body)
            comments.append({
                'body': comment.body,
                'sentiment': sentiment,
                'entities': entities
            })
        posts_data.append({
            'title': post.title + "\nContent:" + "\n" + post.selftext,
            'comments': comments
        })

    return jsonify(posts_data)


@app.route('/fetch_comments', methods=['GET'])
def fetch_comments():
    subreddit_name = request.args.get('subreddit', '')
    subreddit = reddit.subreddit(subreddit_name)
    comments_entities = []

    for post in subreddit.hot(limit=10):
        post.comments.replace_more(limit=0)
        for comment in post.comments.list():
            doc = nlp(comment.body)
            entities = [(ent.text, ent.label_) for ent in doc.ents]
            sentiment = sia.polarity_scores(comment.body)
            comments_entities.append({
                'body': comment.body,
                'sentiment': sentiment,
                'entities': entities
            })

    return jsonify(comments_entities)



if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\l l\Documents\SentimentWatch\venv\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
